In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install chromedriver_py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 18.2 MB/s eta 0:00:00


In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
sudo apt-get install -f

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,134 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,173 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https

In [ ]:
import pandas as pd
import selenium
import pprint

In [ ]:
import sys
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
import pandas
import json
import pprint
from chromedriver_py import binary_path

In [ ]:
chrome_options = webdriver.ChromeOptions()  # Initialize Chrome options
chrome_options.add_argument('--headless')  # Run Chrome in headless mode - In our local notebook we can remove the `--headless` option.
chrome_options.add_argument('--no-sandbox')  # Disable Chrome’s default sandboxing behavior
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument("window-size=1900,800")  # Set the window size for the browser
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")  # Set a custom user agent

In [ ]:
service = Service(executable_path=binary_path)  # Create a Service object with the path to the ChromeDriver executable
wd = webdriver.Chrome(service=service, options=chrome_options)  # Initialize the WebDriver with the specified service and options
wd_detail = webdriver.Chrome(service=service, options=chrome_options)  # Initialize the WebDriver with the specified service and options

In [ ]:
wd.get("https://nextspaceflight.com/launches/past/?page=228&search=")

In [ ]:
from tqdm.notebook import tqdm
import time

launches = []
page = 229

for page in tqdm(range(229, 0, -1)):
    wd.get("https://nextspaceflight.com/launches/past/?page={}&search=".format(page))
    launches_in_a_page = wd.find_elements(By.CSS_SELECTOR, ".mdl-cell.mdl-cell--6-col")
    for launch in launches_in_a_page:
        #show_page()
        launch_detail = {
          'company': "",
          'mission_name': "",
          'launch_date': "",
          'launch_status_color_code': "",
          'detail_url': "",
          'rocket_url': "",
          'page': 0,
          'mission_image': "",
          'mission_detail': ""
          }
        style_content = launch.find_element("tag name", "style").get_attribute("innerHTML")
        launch_detail['mission_image'] = style_content[style_content.find("(")+1:style_content.find(")")]
        launch_info = launch.find_elements(By.CSS_SELECTOR, "div.launch")[0].text.split("\n")
        launch_detail['company'] = launch_info[0]
        launch_detail['mission_name'] = launch_info[1]
        launch_detail['launch_date'] = launch.find_elements(By.CSS_SELECTOR, "div.mdl-card__supporting-text")[0].text.split("\n")[0]
        launch_detail['launch_status_color_code'] = launch.find_elements(By.CSS_SELECTOR, "div.launch")[0].get_attribute("style").split(";")[0].replace("border-color: ","")
        launch_detail['detail_url'] = launch.find_elements(By.CSS_SELECTOR, "a.mdc-button")[0].get_attribute("href")
        wd_detail.get(launch_detail['detail_url'])
        launch_detail['mission_detail'] = wd_detail.find_elements(By.CSS_SELECTOR, "div.mdl-grid.a")[2].text
        rocket_detail_div = wd_detail.find_elements(By.CSS_SELECTOR, "div.mdl-card__actions.mdl-card--border")[0]
        launch_detail['rocket_url'] = rocket_detail_div.find_elements(By.CSS_SELECTOR, "a.mdc-button")[0].get_attribute("href")
        launch_detail['page'] = page
        launches.append(launch_detail)

df_missions = pd.DataFrame(launches)
df_missions

  0%|          | 0/229 [00:00<?, ?it/s]

,company,rocket,mission_name,launch_date,launch_status_color_code,detail_url,rocket_url,page,mission_image,mission_detail
0,RVSN USSR,,Sputnik 8K71PS | Sputnik 1,"Fri Oct 4, 1957 19:28 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/590,https://nextspaceflight.com/rockets/147,229,https://storage.googleapis.com/nextspaceflight...,"Sputnik 1 (Russian: Спутник-1, ""Satellite 1"") ..."
1,US Air Force,,SM-65B Atlas | SCORE,"Thu Dec 18, 1958 23:02 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/858,https://nextspaceflight.com/rockets/173,228,https://storage.googleapis.com/nextspaceflight...,Project SCORE (Signal Communications by Orbiti...
2,ABMA,,Juno II | Pioneer 3,"Sat Dec 6, 1958 05:44 UTC","rgb(255, 153, 0)",https://nextspaceflight.com/launches/details/862,https://nextspaceflight.com/rockets/150,228,https://storage.googleapis.com/nextspaceflight...,"This spacecraft was intended as a lunar probe,..."
3,RVSN USSR,,Vostok | E-1 n°3 (Luna-1),"Thu Dec 4, 1958 18:18 UTC","rgb(218, 52, 50)",https://nextspaceflight.com/launches/details/676,https://nextspaceflight.com/rockets/148,228,https://storage.googleapis.com/nextspaceflight...,The Ye-1 series of soviet lunar probes were th...
4,NASA,,Thor DM-18 Able-I | Pioneer 2,"Sat Nov 8, 1958 07:30 UTC","rgb(218, 52, 50)",https://nextspaceflight.com/launches/details/861,https://nextspaceflight.com/rockets/102,228,https://storage.googleapis.com/nextspaceflight...,Pioneer 2 was the last of the three project Ab...
...,...,...,...,...,...,...,...,...,...,...
6836,CASC,,Long March 6A | Tianhui 5 Group 02,"Thu Jul 4, 2024 22:49 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/7594,https://nextspaceflight.com/rockets/124,1,https://storage.googleapis.com/nextspaceflight...,A pair of satellites officially described as f...
6837,Firefly,,Alpha | Noise of Summer,"Thu Jul 4, 2024 04:04 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/6865,https://nextspaceflight.com/rockets/62,1,https://storage.googleapis.com/nextspaceflight...,CatSat is a 6U CubeSat being built and tested ...
6838,SpaceX,,Falcon 9 Block 5 | Starlink Group 8-9,"Wed Jul 3, 2024 08:55 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/7508,https://nextspaceflight.com/rockets/3,1,https://storage.googleapis.com/nextspaceflight...,51 day turnaround\n338th Falcon recovery attem...
6839,MHI,,H3-22S | ALOS-4,"Mon Jul 1, 2024 03:06 UTC","rgb(69, 207, 93)",https://nextspaceflight.com/launches/details/2554,https://nextspaceflight.com/rockets/105,1,https://storage.googleapis.com/nextspaceflight...,The Advanced Land Observing Satellite-4 (ALOS-...


In [ ]:
## rocket detail
rocket_urls = list(df_missions['rocket_url'].unique())
rockets = []

for rocket_url in tqdm(rocket_urls):
    wd_detail.get(rocket_url)
    rocket_detail = {
          'rocket_url': "",
          'rocket_name': "",
          'rocket_description': ""
          }
    rocket_detail['rocket_url'] = rocket_url
    rocket_detail['rocket_name'] = wd_detail.find_element(By.CSS_SELECTOR, "h3.section--center.mdl-grid").text
    section = wd_detail.find_elements(By.CSS_SELECTOR, "section.card.section--center.white.mdl-grid.mdl-grid--no-spacing.mdl-shadow--6dp")[0]
    rocket_desc = ""
    for text_section in section.find_elements(By.CSS_SELECTOR, "p"):
        rocket_desc = rocket_desc + ' ' + text_section.text
    rocket_detail['rocket_description'] = rocket_desc
    #rocket_detail_list = wd_detail.find_elements(By.CSS_SELECTOR, "div.mdl-card__supporting-text")[0].text.split("\n")[1:]
    #rocket_properties = {item.split(': ')[0]: item.split(': ')[1] for item in rocket_detail_list}
    #rocket_detail = rocket_detail_tmp | rocket_properties
    rockets.append(rocket_detail)

df_rockets = pd.DataFrame(rockets)
df_rockets

  0%|          | 0/197 [00:00<?, ?it/s]

,rocket_url,rocket_name,rocket_description
0,https://nextspaceflight.com/rockets/147,Sputnik 8K71PS,The Sputnik rocket was an uncrewed orbital ca...
1,https://nextspaceflight.com/rockets/173,SM-65B Atlas,"The Convair SM-65B Atlas, or Atlas B, was a p..."
2,https://nextspaceflight.com/rockets/150,Juno II,Juno II was an American space launch vehicle ...
3,https://nextspaceflight.com/rockets/148,Vostok,"Vostok (Russian: Восток, translated as ""East""..."
4,https://nextspaceflight.com/rockets/102,Thor-Able,The Thor-Able was an American expendable laun...
...,...,...,...
192,https://nextspaceflight.com/rockets/292,South Korean ADD Solid-Fuel SLV,South Korean military developed small satelli...
193,https://nextspaceflight.com/rockets/70,Vulcan Centaur,"Vulcan Centaur is a two-stage-to-orbit, heavy..."
194,https://nextspaceflight.com/rockets/294,Gravity 1,4-stage launch vehicle consisting of 5 solid ...
195,https://nextspaceflight.com/rockets/297,KAIROS,4-stage all solid fuel small satellite launch...


In [ ]:
df_rockets.to_excel('rockets.xlsx', index=False)
df_missions.to_excel('missions.xlsx', index=False)